In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [6]:
data = pd.read_excel("Dataset.xlsx")
data

,id,Age,Income,Student,Credit_rating,buys_computer
0,1,<=30,High,No,Fair,No
1,2,<=30,High,No,Excelent,No
2,3,31..40,High,No,Fair,Yes
3,4,>40,Medium,No,Fair,Yes
4,5,>40,Low,Yes,Fair,Yes
5,6,>40,Low,Yes,Excelent,No
6,7,31..40,Low,Yes,Excelent,Yes
7,8,<=30,Medium,No,Fair,No
8,9,<=30,Low,Yes,Fair,Yes
9,10,>40,Medium,Yes,Fair,Yes


In [7]:
data = data.drop(columns = ['id'])

In [9]:
data

,Age,Income,Student,Credit_rating,buys_computer
0,<=30,High,No,Fair,No
1,<=30,High,No,Excelent,No
2,31..40,High,No,Fair,Yes
3,>40,Medium,No,Fair,Yes
4,>40,Low,Yes,Fair,Yes
5,>40,Low,Yes,Excelent,No
6,31..40,Low,Yes,Excelent,Yes
7,<=30,Medium,No,Fair,No
8,<=30,Low,Yes,Fair,Yes
9,>40,Medium,Yes,Fair,Yes


In [8]:
data.dtypes

Age              object
Income           object
Student          object
Credit_rating    object
buys_computer    object
dtype: object

In [10]:
# Preprocess data
def preprocess_dataframe(data):
    # Convert categorical to numerical data
    # Age
    data['Age'] = data['Age'].apply(lambda x: 0 if x=='<=30' else 1 if x=='31..40' else 2)

    # Income
    data['Income'] = data['Income'].apply(lambda x: 0 if x=='High' else 1 if x=='Medium' else 2)

    # Student
    data['Student'] = data['Student'].apply(lambda x: 0 if x=='No' else 1)

    # Credit_rating
    data['Credit_rating'] = data['Credit_rating'].apply(lambda x: 0 if x=='Fair' else 1)

    # Buys_computer
    if 'buys_computer' in data.columns:
        data['buys_computer'] = data['buys_computer'].apply(lambda x: 0 if x=='No' else 1)

        # Set X & y
        y = data['buys_computer'].to_numpy()
        X = data.drop(['buys_computer'], axis=1).to_numpy()
    
        return X, y
    else:
        X = data.to_numpy()
        
        return X, None

# Execute preprocessing data
X, y = preprocess_dataframe(df)
print(X)
print(y)

[[0 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [2 1 0 0]
 [2 2 1 0]
 [2 2 1 1]
 [1 2 1 1]
 [0 1 0 0]
 [0 2 1 0]
 [2 1 1 0]
 [0 1 1 1]
 [1 1 0 1]
 [1 0 1 0]
 [2 1 0 1]]
[0 0 1 1 1 0 1 0 1 1 1 1 1 0]


## 1. Perhitungan probabilitas tiap nilai nominal pada label kelas

In [30]:
def occurence(data):
    labels, counts = np.unique(data, return_counts=True)
    
    prior_data = {}
    for i in range(len(labels)):
        prior_data[labels[i]] = counts[i] / len(data)
        
    return prior_data

prior_data = occurence(y)
print(prior_data)

{0: 0.35714285714285715, 1: 0.6428571428571429}


In [11]:
def occurrence(data):
    labels, counts = np.unique(data, return_counts=True)
    prior_data = {}
    
    for i in range(len(labels)):
        prior_data[labels[i]] = counts[i] / len(data)
        
    return prior_data

prior_data = occurrence(y)
for label, count in prior_data.items():
    print("{:<10} {:.4f}".format(label, count))


0          0.3571
1          0.6429


## 2. Perhitungan probabilitas tiap nilai fitur/variabel prediktor (case) terhadap label kelas

In [12]:
import numpy as np

def occurrence(data):
    labels, counts = np.unique(data, return_counts=True)
    prior_data = {}
    
    for i in range(len(labels)):
        prior_data[labels[i]] = counts[i] / len(data)
        
    return prior_data

def likelihood(X, y):    
    n_class = len(np.unique(y))
    _, n_event = X.shape
    
    likelihood_data = {}
    
    for class_i in range(n_class):
        likelihood_data[class_i] = {}

        for event_j in range(n_event):
            event_data = X[:,event_j][y==class_i]
            proba_event = occurrence(event_data)
        
            likelihood_data[class_i][event_j] = proba_event
            
    return likelihood_data

# Test fungsi
likelihood_data = likelihood(X, y)
for class_i, likelihood_class_i in likelihood_data.items():
    print("Class {}:".format(class_i))
    for event_j, proba_event in likelihood_class_i.items():
        print("Event {}: {}".format(event_j, proba_event))
    print()


Class 0:
Event 0: {0: 0.6, 2: 0.4}
Event 1: {0: 0.4, 1: 0.4, 2: 0.2}
Event 2: {0: 0.8, 1: 0.2}
Event 3: {0: 0.4, 1: 0.6}

Class 1:
Event 0: {0: 0.2222222222222222, 1: 0.4444444444444444, 2: 0.3333333333333333}
Event 1: {0: 0.2222222222222222, 1: 0.4444444444444444, 2: 0.3333333333333333}
Event 2: {0: 0.3333333333333333, 1: 0.6666666666666666}
Event 3: {0: 0.6666666666666666, 1: 0.3333333333333333}



In [13]:
def occurrence(data):
    labels, counts = np.unique(data, return_counts=True)
    prior_data = {}
    
    for i in range(len(labels)):
        prior_data[labels[i]] = counts[i] / len(data)
        
    return prior_data

def evidence(X):
    _, n_event = X.shape
        
    evidence_data = {}
    for event_i in range(n_event):
        event_data = X[:, event_i]
        proba_event = occurrence(event_data)
        
        evidence_data[event_i] = proba_event
        
    return evidence_data

# Test fungsi
evidence_data = evidence(X)
for event_i, proba_event in evidence_data.items():
    print("Event {}: {}".format(event_i, proba_event))


Event 0: {0: 0.35714285714285715, 1: 0.2857142857142857, 2: 0.35714285714285715}
Event 1: {0: 0.2857142857142857, 1: 0.42857142857142855, 2: 0.2857142857142857}
Event 2: {0: 0.5, 1: 0.5}
Event 3: {0: 0.5714285714285714, 1: 0.42857142857142855}


### 3. Perkalian tiap probabilitas nilai fitur/variabel prediktor terhadap label kelas dengan probabilitas tiap nilai nominal pada label kelas

In [17]:
# Buat Dataset testing
data_test = {'Age': ['<=30', '<=30'],
             'Income': ['Medium', 'Medium'],
             'Student': ['Yes', 'No'],
             'Credit_rating': ['Fair', 'Excelent']}

# Buat dataframe
df_test = pd.DataFrame(data_test)

# Print dataframe
df_test

,Age,Income,Student,Credit_rating
0,<=30,Medium,Yes,Fair
1,<=30,Medium,No,Excelent


In [18]:
def evidence(X):
    _, n_event = X.shape
        
    evidence_data = {}
    for event_i in range(n_event):
        event_data = X[:, event_i]
        proba_event = occurrence(event_data)
        
        evidence_data[event_i] = proba_event
        
    return evidence_data

# Test fungsi
evidence_data = evidence(X)
for event_i, proba_event in evidence_data.items():
    print("Event {}: {}".format(event_i, proba_event))

Event 0: {0: 0.35714285714285715, 1: 0.2857142857142857, 2: 0.35714285714285715}
Event 1: {0: 0.2857142857142857, 1: 0.42857142857142855, 2: 0.2857142857142857}
Event 2: {0: 0.5, 1: 0.5}
Event 3: {0: 0.5714285714285714, 1: 0.42857142857142855}


In [19]:
# Preprocess data test
X_test, _ = preprocess_dataframe(df_test)

print(X_test)

[[0 1 1 0]
 [0 1 0 1]]


In [21]:
def predict(X_test, prior, likelihood, evidence):
    n_class = len(prior)
    n_data, n_event = X_test.shape
    
    # Buat posterior
    posterior_data = {}
    for data_i in range(n_data):
        posterior_data[data_i] = {}
        
        for class_j in range(n_class):
            posterior_data[data_i][class_j] = {}

            tot_likelihood = 1
            tot_evidence = 1
            for event_k in range(n_event):
                tot_likelihood *= likelihood[class_j][event_k][X_test[data_i][event_k]]
                tot_evidence *= evidence[event_k][X_test[data_i][event_k]]

            posterior = (tot_likelihood/tot_evidence) * prior[class_j]
            posterior_data[data_i][class_j] = posterior
    
    # Normalize
    for data_j in range(n_data):
        sum_prob = 0
        for class_i in range(n_class):
            sum_prob += posterior_data[data_j][class_i]
            
        for class_i in range(n_class):
            posterior_data[data_j][class_i] /= sum_prob
    
    # Hasil prediksi
    placeholder_class = np.zeros((n_data, n_class))
    for class_i in range(n_class):
        for data_j in range(n_data):
            placeholder_class[data_j][class_i] = posterior_data[data_j][class_i]
    
    choosen_class = np.argmax(placeholder_class, axis=1)

    return choosen_class, posterior_data


# Test fungsi
choosen_class, posterior_data = predict(X_test, prior_data, likelihood_data, evidence_data)

# Tampilkan output
print("Predicted class: {}".format(choosen_class))
for data_i in range(len(X_test)):
    print("Data {}: {}".format(data_i, posterior_data[data_i]))


Predicted class: [1 0]
Data 0: {0: 0.1954947707160097, 1: 0.8045052292839903}
Data 1: {0: 0.8536299765807963, 1: 0.14637002341920374}


### 4.mebandingkan nilai probabilitas yang besar sebagai hasil prediksi label kelas dari record/instance 

In [16]:
def preprocess_dataframe_invert(data):
    # Convert categorical to numerical data
    # Age
    data['Age'] = data['Age'].apply(lambda x: 0 if x=='<=30' else 1 if x=='31..40' else 2)

    # Income
    data['Income'] = data['Income'].apply(lambda x: 0 if x=='High' else 1 if x=='Medium' else 2)

    # Student
    data['Student'] = data['Student'].apply(lambda x: 0 if x=='No' else 1)

    # Credit_rating
    data['Credit_rating'] = data['Credit_rating'].apply(lambda x: 0 if x=='Fair' else 1)
    
    # Target
    data['buys_computer'] = data['buys_computer'].apply(lambda x: 'N' if x==0 else 'Y')
    
    return data

# Execute preprocessing data
df_test['buys_computer'] = choosen_class

df_test = preprocess_dataframe_invert(df_test)
df_test

,Age,Income,Student,Credit_rating,buys_computer
0,2,2,1,1,Y
1,2,2,1,1,N


### 5.  Pembagian data training dan data testing menggunakan percentage split (misal: 70:30, 80:20, dan 90:10) xxxx

In [22]:
import pandas as pd
import numpy as np
import math

def train_test_split(df, test_size=0.2, random_state=None):
    # Membuat array indeks acak untuk data
    np.random.seed(random_state)
    shuffled_indices = np.random.permutation(len(df))
    # Menghitung jumlah data testing berdasarkan test_size
    test_size = math.ceil(len(df) * test_size)
    
    # Memilih indeks data training dan data testing
    test_indices = shuffled_indices[:test_size]
    train_indices = shuffled_indices[test_size:]
    
    # Membagi data menjadi data training dan data testing
    train_data = df.iloc[train_indices]
    test_data = df.iloc[test_indices]
    
    return train_data, test_data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=1)
X_test = test_data.iloc[:,:-1].values
Y_test = test_data.iloc[:,-1].values
print("Training data :", train_data)
print("\n", "Testing data :", test_data)

Training data :     Age  Income  Student  Credit_rating  buys_computer
2     1       0        0              0              1
10    0       1        1              1              1
4     2       2        1              0              1
1     0       0        0              1              0
12    1       0        1              0              1
0     0       0        0              0              0
13    2       1        0              1              0
9     2       1        1              0              1
8     0       2        1              0              1
11    1       1        0              1              1
5     2       2        1              1              0

 Testing data :    Age  Income  Student  Credit_rating  buys_computer
3    2       1        0              0              1
7    0       1        0              0              0
6    1       2        1              1              1


### 6. Perhitungan performa klasifikasi menggunakan confusion matrix (Accuracy, Recall, Precision, F-measure)

In [23]:
def prob_prior(df, Y):
    label_kelas = sorted(list(df[Y].unique()))
    prior = []
    for i in label_kelas:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

def likelihood_categorical(df, nama_var, nilai_var, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_dgsyrt_y = len(df[df[nama_var]==nilai_var]) / len(df)
    return p_x_dgsyrt_y

def naive_bayes_categorical(df, X, Y):
    # mengambil nama kolom
    nama_kolom = list(df.columns)[:-1]
    # menghitung prob prior
    prior = prob_prior(df, Y)
    Y_pred = []
    # lopping pada setiap baris data
    for x in X:
        # menghitung likelihood
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(nama_kolom)):
                likelihood[j] *= likelihood_categorical(df, nama_kolom[i], x[i], Y, labels[j])
                                                                                           
        # menghitung posterior probability
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        Y_pred.append(np.argmax(post_prob))
    return np.array(Y_pred)

In [24]:
Y_pred = naive_bayes_categorical(train_data, X=X_test, Y="buys_computer")

In [25]:
# Menghitung true positives, false positives, true negatives, false negatives
tp = fp = tn = fn = 0
for i in range(len(Y_test)):
    if Y_test[i] == Y_pred[i] == 1:
        tp += 1
    elif Y_test[i] == Y_pred[i] == 0:
        tn += 1
    elif Y_test[i] == 0 and Y_pred[i] == 1:
        fp += 1
    else:
        fn += 1

# Menghitung accuracy, recall, precision, dan f1-score
accuracy = (tp + tn) / (tp + fp + tn + fn)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = 2 * (precision * recall) / (precision + recall)

# Menampilkan confusion matrix dan performa klasifikasi
print(f"Confusion Matrix:\n{tn}\t{fp}\n{fn}\t{tp}\n")
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Confusion Matrix:
0	1
0	2

Accuracy: 0.67
Recall: 1.00
Precision: 0.67
F1 Score: 0.80


In [26]:
# Menghitung true positives, false positives, true negatives, false negatives
tp = fp = tn = fn = 0
for i in range(len(Y_test)):
    if Y_test[i] == 1 and Y_pred[i] == 1:
        tp += 1
    elif Y_test[i] == 1 and Y_pred[i] == 0:
        fn += 1
    elif Y_test[i] == 0 and Y_pred[i] == 1:
        fp += 1
    elif Y_test[i] == 0 and Y_pred[i] == 0:
        tn += 1

# Menghitung accuracy, recall, precision, dan f1-score
accuracy = (tp + tn) / (tp + fp + tn + fn)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = 2 * (precision * recall) / (precision + recall)

# Menampilkan confusion matrix dan performa klasifikasi
print(f"Confusion Matrix:\n{tn}\t{fp}\n{fn}\t{tp}\n")
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1_score:.2f}")


Confusion Matrix:
0	1
0	2

Accuracy: 0.67
Recall: 1.00
Precision: 0.67
F1 Score: 0.80
